# Лабораторная работа №6
## Поиск экстремума функции многих переменных
*В данной работе был выполнен поиск экстремума функции нескольких переменных с помощью шрадиентного
метода с дроблением шага*

*Суть метода состоит в выборе параметра $\alpha>0$ и параметра дробления $0<\lambda<1$
Затем при выбранном начальном приближении ищется градиент от заданной функции и проверяется условие
$f(x^k+\alpha p^k)<f(x^k)$, где p-веткор градиента, если условие выполнено, то точка $x^k+\alpha p^k$
берётся за следующее приближение, иначе изменяем(дробим) шаг $\alpha$, домножая его на параметр $\lambda$. Условие выхода из цикла $\frac{||x^k-x^{k-1}||}{1+||x^{k-1}||}<\delta$*

*Также был построен график поверхности, линии уровня и траектория поиска и оценена овражность.
Овражность оценивалась как отношение собственных чисел матрицы Гессе.
$$H(f)={\begin{bmatrix}{\frac  {\partial ^{2}f}{\partial x_{1}^{2}}}&{\frac  {\partial ^{2}f}{\partial x_{1}\,\partial x_{2}}}&\cdots &{\frac  {\partial ^{2}f}{\partial x_{1}\,\partial x_{n}}}\\\\{\frac  {\partial ^{2}f}{\partial x_{2}\,\partial x_{1}}}&{\frac  {\partial ^{2}f}{\partial x_{2}^{2}}}&\cdots &{\frac  {\partial ^{2}f}{\partial x_{2}\,\partial x_{n}}}\\\\\vdots &\vdots &\ddots &\vdots \\\\{\frac  {\partial ^{2}f}{\partial x_{n}\,\partial x_{1}}}&{\frac  {\partial ^{2}f}{\partial x_{n}\,\partial x_{2}}}&\cdots &{\frac  {\partial ^{2}f}{\partial x_{n}^{2}}}\end{bmatrix}}$$*

*Кроме того была обеспечена работа программы в режиме "чёрного ящика" и для функций размерности до 10*

Работу выполнил: студент 431 группы Перекалов Александр

In [11]:
import matplotlib.pyplot as plt
def func1(L=[]):
    #f=x**4+y**4+2*(x**2)*(y**2)-4*x+3
    f=L[0]**4+L[1]**4+2*(L[0]**2)*(L[1]**2)-4*L[0]+3
    return f
L=[2,4]
print(func1(L))
#print(func1(2,4))

395


In [3]:
#обьявление многомерной функции
import math
import numpy
N=3
def func2(L):
    f=1
    for i in range(0,len(L)):
        f=f*math.exp(-0.2*(L[i]-2)**2)
    return f
L=[-0.145,-0.5,-0.145]
y=func2(L)
print(y)     

0.04548330242723231


In [62]:
#график поверхности
from mpl_toolkits.mplot3d import Axes3D
import numpy
fig = plt.figure()
ax = Axes3D(fig)
X=numpy.arange(-2,2,0.2)
Y=numpy.arange(-2,2,0.2)
Xg, Yg = numpy.meshgrid(X, Y)
Z=func1(numpy.meshgrid(X,Y))
ax.plot_surface(Xg, Yg, Z, rstride=1, cstride=1, cmap='Spectral')
plt.show()

In [5]:
import numpy
import scipy.misc
Poi=[0,2]
#Poi=[1,1.5,2,2.5,1,4,2,6,8,10]
def gradient(func, point):#point-список(точка с координатами)
    grad=[]
    args = point
    for i in range(0,len(point)):
        def wraps(x):
            args[i] = x
            return func(args)
            #return func(*args)#для изначальной функции
        grad.append(scipy.misc.derivative(wraps, point[i], dx = 1e-6))
    return grad
#h=gradient(func1, Poi)
h=gradient(func2, Poi)
print(h)

[0.35946317128443539, 0.0]


In [13]:
#метод для двумерной функции
import numpy
delta=0.001
Xk=[1,3]#начальное приближение
S=[0.,0.]#следующее приближение
PLOTX=[Xk[0]]#список для построения пути поиска
PLOTY=[Xk[1]]
n=0
def gradmetod(func1, Xk, delta, maxiter, alpha=()):
    n=0
    lambd=0.5#параметр дробления
    while(True):
        step=0.8
        Pk=gradient(func1,Xk)
        n=n+1
        if n>maxiter:
            print("превышено число итераций")
            break
        while(True):
            S1=0
            S2=0
            for i in range(0,len(S)):
                S1=Xk[i]
                S2=step*Pk[i]
                S[i]=S1-S2
            if func1(S)>func1(Xk):
                step=step*lambd
            else:
                PLOTX.append(S[0])
                PLOTY.append(S[1])
                break
        if numpy.linalg.norm(numpy.array(S)-numpy.array(Xk))/(1+numpy.linalg.norm(numpy.array(Xk)))>delta:
            for j in range(0,len(S)):
                Xk[j]=S[j]
        else:
            break
    print(n)
    return(S)
S=gradmetod(func1,Xk,delta,1000)
Gess=numpy.zeros([2,2], "f")
Gess[0][0]=12*(S[0]**2)+4*(S[1]**2)
Gess[0][1]=8*S[0]*S[1]
Gess[1][0]=Gess[0][1]
Gess[1][1]=12*(S[1]**2)+4*(S[0]**2)
z1=numpy.linalg.eigvals(Gess)
if abs(z1[0])>abs(z1[1]):
    ovr=abs(z1[0]/z1[1])
else:
    ovr=abs(z1[1]/z1[0])
print("овражность", ovr)
print(S)

6
овражность 3.0
[0.99993103365575631, 2.3765232565980905e-06]


In [91]:
#построение линий уровня
X=numpy.arange(-2.1,2.1,0.1)
Y=numpy.arange(-2.1,2.1,0.1)
L=[0,0.1,0.3,0.5,1,1.5,2,2.5,3,4,5,6,7,8,9,10,12,14,16,18,20,22,24,26,28,30]
#X, Y = numpy.meshgrid(X, Y)
Xg, Yg = numpy.meshgrid(X, Y)
Z=func1(numpy.meshgrid(X,Y))
#Z=numpy.zeros([len(X), len(X)], "f")
#Z=func1(X,Y)
plt.figure()
cp = plt.contour(Xg, Yg, Z, levels=L)
manual_locations=[(1,1)]
plt.plot(PLOTX,PLOTY, 'r')
plt.show()
#print(X)

In [9]:
import numpy
Xk=[1.5,1.2,1.4,2,2.5,3,1,4,2.5,2]#начальное приближение
S=[0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]#следующее приближение
def gradmetod2(func2, Xk, delta, maxiter, alpha=()):
    n=0
    lambd=0.5#параметр дробления
    step=2
    while(True):
        Pk=gradient(func2,Xk)
        n=n+1
        if n>maxiter:
            print("превышено число итераций")
            break
        while(True):
            S1=0
            S2=0
            for i in range(0,len(S)):
                S1=Xk[i]
                S2=step*Pk[i]
                S[i]=S1+S2
            if func2(S)<func2(Xk):
                step=step*lambd
            else:
                break
        if numpy.linalg.norm(numpy.array(S)-numpy.array(Xk))/(1+numpy.linalg.norm(numpy.array(Xk)))>delta:
            for j in range(0,len(S)):
                Xk[j]=S[j]
        else:
            break
    print(n)
    return S
numpy.set_printoptions(precision=4,suppress=True, linewidth=250)
print(gradmetod2(func2, Xk, 0.001,1000))

8
[1.9999107166108221, 1.9998563965709895, 1.9998926098198555, 2.000001250787701, 2.000091784770289, 2.0001823188083883, 1.9998201826837452, 2.0003633867458088, 2.0000917849368225, 2.0000012508432121]
